In [ ]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

Loading training and testing *set*

In [ ]:
(training_images1, training_labels1) , (validation_images, validation_labels) = tf.keras.datasets.cifar10.load_data()

Loading training and testing from robust dataset and preprocess

In [ ]:
train_data = ch.cat(ch.load(os.path.join( f"drive/MyDrive/CIFAR_ims")))
train_labels = ch.cat(ch.load(os.path.join( f"drive/MyDrive/CIFAR_lab")))
train_data.shape

torch.Size([50000, 3, 32, 32])

In [ ]:
test_data = np.load('drive/MyDrive/rob5.npy')
test_label = np.load('drive/MyDrive/rob.l5.npy')
test_data.shape

(10000, 3, 32, 32)

trian model on robust dataset

In [ ]:
checkpoint_path = "training_1/cp1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
EPOCHS = 5
history = model.fit(train_data, training_labels, epochs=EPOCHS,
          callbacks=[cp_callback], validation_data = (valid_X, validation_labels), batch_size=64)

Epoch 1/5
782/782 [==============================] - ETA: 0s - loss: 0.7425 - accuracy: 0.7442
Epoch 1: saving model to training_1/cp1.ckpt
782/782 [==============================] - 577s 697ms/step - loss: 0.7425 - accuracy: 0.7442 - val_loss: 0.5350 - val_accuracy: 0.8183
Epoch 2/5
782/782 [==============================] - ETA: 0s - loss: 0.3264 - accuracy: 0.8878
Epoch 2: saving model to training_1/cp1.ckpt
782/782 [==============================] - 548s 700ms/step - loss: 0.3264 - accuracy: 0.8878 - val_loss: 0.4872 - val_accuracy: 0.8350
Epoch 3/5
782/782 [==============================] - ETA: 0s - loss: 0.1532 - accuracy: 0.9497
Epoch 3: saving model to training_1/cp1.ckpt
782/782 [==============================] - 535s 685ms/step - loss: 0.1532 - accuracy: 0.9497 - val_loss: 0.6146 - val_accuracy: 0.8134
Epoch 4/5
782/782 [==============================] - ETA: 0s - loss: 0.0582 - accuracy: 0.9836
Epoch 4: saving model to training_1/cp1.ckpt
782/782 [==========================

Importing model trained on robust dataset

In [ ]:
checkpoint_path = "training_1/cp1.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp1.ckpt.index', 'cp1.ckpt.data-00000-of-00001']

In [ ]:
model_r.load_weights(checkpoint_path)

Importing model trained on standard dataset

In [ ]:
checkpoint_path = "training_3/cp3.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [ ]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp3.ckpt.data-00000-of-00001', 'cp3.ckpt.index']

In [ ]:
model.load_weights(checkpoint_path)

In [ ]:
loss, accuracy = model.evaluate(valid_X, validation_labels, batch_size=64)

157/157 [==============================] - 36s 163ms/step - loss: 0.1740 - accuracy: 0.9552


In [ ]:
logits_model = tf.keras.Model(model.input,model.layers[-1].output)

In [ ]:
logits_model_r = tf.keras.Model(model_r.input,model_r.layers[-1].output)

robust/normal

Define pgd attack perturbed images from standard dataset with standard attack and classify with standard model.

In [ ]:
import tensorflow as tf
import numpy as np
# Define the PGD attack function
def pgd_attack(image, model, epsilon, num_steps, step_size):
    perturbed_image = tf.identity(image)

    for _ in range(num_steps):
        with tf.GradientTape() as tape:
            tape.watch(perturbed_image)
            predictions = model(perturbed_image)
            loss = tf.keras.losses.sparse_categorical_crossentropy(training_labels1[random_index], predictions)

        gradient = tape.gradient(loss, perturbed_image)
        signed_grad = tf.sign(gradient)
        perturbed_image = perturbed_image + step_size * signed_grad
        perturbed_image = tf.clip_by_value(perturbed_image, image - epsilon, image + epsilon)
        perturbed_image = tf.clip_by_value(perturbed_image, 0, 255)

    return perturbed_image

num_correct = 0

for j in range(100):
    random_index = np.random.randint(validation_images.shape[0])

    preprocessed_image = preprocess_image_input(training_images1[random_index])
    reshaped_image = np.reshape(preprocessed_image, (1, 32, 32, 3))

    perturbed_image = pgd_attack(reshaped_image, model, epsilon, num_steps, step_size)
    perturbed_image = np.squeeze(perturbed_image, axis=0)

    # Make predictions
    predictions = model.predict(perturbed_image[np.newaxis, ...])

    # Get predicted label
    predicted_label = np.argmax(predictions[0])

    # Compare with true label
    true_label = np.reshape(training_labels1[random_index], (1,))[0]
    if predicted_label == true_label:
        num_correct += 1

accuracy = num_correct / 100
print("Accuracy: {:.2%}".format(accuracy))


1/1 [==============================] - 0s 24ms/step
Accuracy: 13.00%


Define pgd attack perturbed images from standard dataset with robust attack and classify with robust model.

In [ ]:
import tensorflow as tf
import numpy as np

# Define the PGD attack function
def pgd_attack(image, model, epsilon, num_steps, step_size):
    perturbed_image = tf.identity(image)

    for _ in range(num_steps):
        with tf.GradientTape() as tape:
            tape.watch(perturbed_image)
            predictions = model(perturbed_image)
            loss = tf.keras.losses.sparse_categorical_crossentropy(training_labels1[random_index], predictions)

        gradient = tape.gradient(loss, perturbed_image)
        signed_grad = tf.sign(gradient)
        perturbed_image = perturbed_image + step_size * signed_grad
        perturbed_image = tf.clip_by_value(perturbed_image, image - epsilon, image + epsilon)
        perturbed_image = tf.clip_by_value(perturbed_image, 0, 255)

    return perturbed_image

num_correct = 0

for j in range(100):
    random_index = np.random.randint(validation_images.shape[0])

    preprocessed_image = preprocess_image_input(training_images1[random_index])
    reshaped_image = np.reshape(preprocessed_image, (1, 32, 32, 3))

    perturbed_image = pgd_attack(reshaped_image, model_r, epsilon, num_steps, step_size)
    perturbed_image = np.squeeze(perturbed_image, axis=0)

    # Make predictions
    predictions = model_r.predict(perturbed_image[np.newaxis, ...])

    # Get predicted label
    predicted_label = np.argmax(predictions[0])

    # Compare with true label
    true_label = np.reshape(training_labels1[random_index], (1,))[0]
    if predicted_label == true_label:
        num_correct += 1

accuracy = num_correct / 100
print("Accuracy: {:.2%}".format(accuracy))

1/1 [==============================] - 0s 23ms/step
Accuracy: 21.00%


Define pgd attack perturbed images from standard dataset with standard attack and classify with robust model.

In [ ]:
import tensorflow as tf
import numpy as np

# Define the PGD attack function
def pgd_attack(image, model, epsilon, num_steps, step_size):
    perturbed_image = tf.identity(image)

    for _ in range(num_steps):
        with tf.GradientTape() as tape:
            tape.watch(perturbed_image)
            predictions = model(perturbed_image)
            loss = tf.keras.losses.sparse_categorical_crossentropy(training_labels1[random_index], predictions)

        gradient = tape.gradient(loss, perturbed_image)
        signed_grad = tf.sign(gradient)
        perturbed_image = perturbed_image + step_size * signed_grad
        perturbed_image = tf.clip_by_value(perturbed_image, image - epsilon, image + epsilon)
        perturbed_image = tf.clip_by_value(perturbed_image, 0, 255)

    return perturbed_image

num_correct = 0

for j in range(100):
    random_index = np.random.randint(validation_images.shape[0])

    preprocessed_image = preprocess_image_input(training_images1[random_index])
    reshaped_image = np.reshape(preprocessed_image, (1, 32, 32, 3))

    perturbed_image = pgd_attack(reshaped_image, model, epsilon, num_steps, step_size)
    perturbed_image = np.squeeze(perturbed_image, axis=0)

    # Make predictions
    predictions = model_r.predict(perturbed_image[np.newaxis, ...])

    # Get predicted label
    predicted_label = np.argmax(predictions[0])

    # Compare with true label
    true_label = np.reshape(training_labels1[random_index], (1,))[0]
    if predicted_label == true_label:
        num_correct += 1

accuracy = num_correct / 100
print("Accuracy: {:.2%}".format(accuracy))

1/1 [==============================] - 0s 22ms/step
Accuracy: 33.00%


Define pgd attack perturbed images from robust dataset with standard attack and classify with robust model.

In [ ]:
import tensorflow as tf
import numpy as np


# Define the PGD attack function
def pgd_attack(image, model, epsilon, num_steps, step_size):
    perturbed_image = tf.identity(image)

    for _ in range(num_steps):
        with tf.GradientTape() as tape:
            tape.watch(perturbed_image)
            predictions = model(perturbed_image)
            loss = tf.keras.losses.sparse_categorical_crossentropy(training_labels[random_index], predictions)

        gradient = tape.gradient(loss, perturbed_image)
        signed_grad = tf.sign(gradient)
        perturbed_image = perturbed_image + step_size * signed_grad
        perturbed_image = tf.clip_by_value(perturbed_image, image - epsilon, image + epsilon)
        perturbed_image = tf.clip_by_value(perturbed_image, 0, 255)

    return perturbed_image

num_correct = 0

for j in range(100):
    random_index = np.random.randint(validation_images.shape[0])

    preprocessed_image = preprocess_image_input(training_images[random_index])
    reshaped_image = np.reshape(preprocessed_image, (1, 32, 32, 3))

    perturbed_image = pgd_attack(reshaped_image, model, epsilon, num_steps, step_size)
    perturbed_image = np.squeeze(perturbed_image, axis=0)

    # Make predictions
    predictions = model_r.predict(perturbed_image[np.newaxis, ...])

    # Get predicted label
    predicted_label = np.argmax(predictions[0])

    # Compare with true label
    true_label = np.reshape(training_labels[random_index], (1,))[0]
    if predicted_label == true_label:
        num_correct += 1

accuracy = num_correct / 100
print("Accuracy: {:.2%}".format(accuracy))

1/1 [==============================] - 0s 26ms/step
Accuracy: 64.00%
